In [72]:
"""new line in ingredients
"""

'new line in ingredients\n'

In [73]:
import os
import shutil
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import json

In [74]:
def getingredients(soup):
    ingredients=soup.find_all('ul',{'class':"ingredient-list"})
    ing=""
    for ingredient in ingredients:
        for li in ingredient.find_all('li'):
            if li.find('h4'):
                #current_heading = li.find('h4').text.strip()  # update the current heading
                continue
            ingredient_text = ''
            for span in li.find_all('span', {'class': 'ingredient-text'}):
                #ingredient_text += span.text.strip()
                ingredient_text += span.get_text(separator=' ', strip=True) + ' '
            for quantity in li.find_all('span', {'class': 'ingredient-quantity'}):
                ingredient_text = quantity.text.strip() + " " + ingredient_text
            ing+=ingredient_text.strip()+"\n"
    return ing

In [75]:
#print(getingredients(soup))

In [76]:
def getcookingtime(soup):
    duration=""
    c_time=soup.find_all('div',{'class':"facts"})
    for c in c_time:
        c1=c.find('div',{'class':"facts__item"})
        cooking_time=c1.find('dd',{'class':"facts__value"})
        duration+=cooking_time.text
    return duration    

In [77]:
#print(getcookingtime(soup))

In [78]:
def getdirections(soup):
    layout=soup.find('div',{'class':"current-recipe"})
    u_list=layout.find('ul',{'class':"direction-list"})
    list_items=u_list.find_all('li',{'class':"direction"})
    directions=""
    for list_item in list_items:
        directions+=list_item.text + "\n"
    return directions
#print(getdirections(soup))
    

In [79]:
def getcalories(soup):
    calories = soup.find('script', type='application/ld+json').string
    calories = json.loads(calories)["nutrition"]["calories"]
    return calories

In [80]:
def getservings(soup):
    serving=""
    sv=soup.find_all('div',{'class':"facts"})
    for s in sv:
        c1=s.find_all('div',{'class':"facts__item"})
        for c in c1:
            serve=c.find('dd',{'class':"facts__value"})
            #serve_find=serve.find('div',{'class':"adjust"})
            if(serve.find('div',{'class':"adjust"})):
                serve_count=serve.find('div',{'class':"adjust"}).find('span',{'class':"value"}).text
                serving+=serve_count
                return serving
            else:
                continue    
#print(getservings(soup))            

In [81]:
def getservings(soup):
    serving=""
    sv=soup.find_all('div',{'class':"facts"})
    for s in sv:
        c1=s.find_all('div',{'class':"facts__item"})
        for c in c1:
            serve=c.find('dd',{'class':"facts__value"})
            #serve_find=serve.find('div',{'class':"adjust"})
            if(serve.find('span',{'class':"value"})):
                serve_count=serve.find('span',{'class':"value"}).text.strip()[0]
                serving+=serve_count
            else:
                continue 
    return serve_count

Encoidng main function

In [82]:
def scrape(name,url):
    page=requests.get(url)
    soup=BeautifulSoup(page.content,'html.parser')
    dict={'name': name, 'cooking_time': getcookingtime(soup), 'calories': getcalories(soup), 'servings': getservings(soup),
             'ingredients': getingredients(soup), 'directions': getdirections(soup)}
    return dict
    

In [88]:
def main():
    links=pd.read_csv(r"C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\links.csv")
    name=links["name"]
    url=links["link"]
    recipe_info=[]
    for item in range(2):
        recipe_info.append(scrape(name[item],url[item]))
    #return recipe_info
    
    with open('recipes.json', 'r+') as f:             # write the scraped data to a JSON file
        json.dump(recipe_info, f)
        
if __name__ == "__main__":
    main()
    

In [89]:
with open('recipes.json','r') as d:
    firstline=d.readline()
    print(firstline)

[{"name": "Chicken Tikka Masala", "cooking_time": "2hrs", "calories": "499.1", "servings": "4", "ingredients": "1 1\u20442 lbs boneless skinless chicken , cut in 1 inch cubes\n1 cup plain yogurt\n2 tablespoons lemon juice\n2 teaspoons ground cumin\n2 teaspoons    ground red pepper\n2 teaspoons black pepper\n1 teaspoon cinnamon\n1 teaspoon salt\n1 piece    minced ginger (1-inch long)\n6 bamboo skewers (6-inch)\n1 tablespoon unsalted butter\n2 garlic cloves , minced\n1 jalapeno chile , minced\n2 teaspoons ground coriander\n1 teaspoon ground cumin\n1 teaspoon paprika\n1 teaspoon garam masala (buy in Indian market)\n1\u20442 teaspoon salt\n1 (8   ounce) can tomato sauce\n1 cup    whipping cream\n1\u20444 cup   chopped fresh cilantro\n", "directions": "Soak bamboo skewers in water.\nThread chicken on skewers, and marinate (in the refrigerator) for an hour or so.\nDiscard marinade.\nFor sauce, melt butter on medium heat.\nAdd garlic & jalapeno; cook 1 minute.\nStir in coriander, cumin, papri